# Importação de Bibliotecas

In [1]:
!pip install UTide

In [2]:
import os
import pandas as pd
import warnings
import pickle

import numpy as np
import matplotlib.pyplot as plt
from utide import ut_constants

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/utide/harmonics.py:16: RuntimeWarning: invalid value encountered in cast
  nshallow = np.ma.masked_invalid(const.nshallow).astype(int)
/usr/local/lib/python3.10/dist-packages/utide/harmonics.py:17: RuntimeWarning: invalid value encountered in cast
  ishallow = np.ma.masked_invalid(const.ishallow).astype(int) - 1


# Dados de Corrente

In [3]:
odas_corrente = pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Boia ODAS/Corrente/Corrente.csv")
cattalini_corrente = pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Cattalini Correntometro/Cattalini Correntometro.csv")

In [4]:
odas_corrente.drop(['Temperatura da Água', 'Pressão', 'Dir.Corrente Rio', 'Tábua'], axis=1, inplace=True)
cattalini_corrente.drop(['Temperatura da Água', 'Pressão', 'Dir.Corrente Rio', 'Tábua'], axis=1, inplace=True)

In [5]:
odas_corrente.reset_index(drop=True, inplace=True)
cattalini_corrente.reset_index(drop=True, inplace=True)

In [6]:
odas_corrente["Datetime"] = pd.to_datetime(odas_corrente["Data/Hora"], utc=True).dt.round('ms')
cattalini_corrente["Datetime"] = pd.to_datetime(cattalini_corrente["Data/Hora"], utc=True).dt.round('ms')

odas_corrente.drop(["Data/Hora"], axis=1, inplace=True)
cattalini_corrente.drop(["Data/Hora"], axis=1, inplace=True)

odas_corrente['Datetime'] = odas_corrente['Datetime'].astype('datetime64[ms, UTC]')
cattalini_corrente['Datetime'] = cattalini_corrente['Datetime'].astype('datetime64[ms, UTC]')

odas_corrente = odas_corrente[["Datetime"] + [col for col in odas_corrente.columns if col != "Datetime"]]
cattalini_corrente = cattalini_corrente[["Datetime"] + [col for col in cattalini_corrente.columns if col != "Datetime"]]

# Dados Meteorologia

In [7]:
odas_meteorologia =  pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Boia ODAS/Meteorologia/Meteorologia.csv")
cattalini_meteorologia = pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Cattalini Meteorologia/Cattalini Meteorologia.csv")

In [8]:
odas_meteorologia.drop(['Temp.', 'Sens.Term.', 'Umid.Rel.', 'Pressão', 'Pressão Tend.', 'Estação', 'DH Registro'], axis=1, inplace=True)
cattalini_meteorologia.drop(['Temp.', 'Sens.Term.', 'Umid.Rel.', 'Pressão', 'Pressão Tend.', 'Estação', 'DH Registro'], axis=1, inplace=True)

In [9]:
cattalini_meteorologia.columns = ["Datetime", "Velocidade do Vento", "Rajada de Vento", "Direção do Vento", "Setor do Vento"]
odas_meteorologia.columns = ["Datetime", "Velocidade do Vento", "Rajada de Vento", "Direção do Vento", "Setor do Vento"]

In [10]:
cattalini_meteorologia["Datetime"] = pd.to_datetime(cattalini_meteorologia["Datetime"], utc=True).dt.round('ms')
odas_meteorologia["Datetime"] = pd.to_datetime(odas_meteorologia["Datetime"], utc=True).dt.round('ms')

cattalini_meteorologia["Datetime"] = pd.to_datetime(cattalini_meteorologia["Datetime"]).astype('datetime64[ms, UTC]')
odas_meteorologia["Datetime"] = pd.to_datetime(odas_meteorologia["Datetime"]).astype('datetime64[ms, UTC]')

# Dados Marégrafos

In [11]:
maregrafo = pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Maregrafos/Maregrafos.csv")

In [12]:
maregrafo["Tábua"].replace({
    "Catalini": "Cattalini",
    "Marégrafo - Cobras": "Ilha das Cobras",
    "Marégrafo - Félix" : "Ponta do Félix",
    "Marégrafo - Galheta" : "Canal da Galheta",
    'Marégrafo - Jererê' : 'Jererê',
    "Marégrafo - Porto" : "Cais Oeste",
    "TCP - Cais Leste" : "Cais Leste",
}, inplace=True)

maregrafo.drop(["BM/PM"], axis=1, inplace=True)
maregrafo.columns = ['Tábua', 'Datetime', 'Altura']
maregrafo["Datetime"] = pd.to_datetime(maregrafo["Datetime"], utc=True).dt.round('ms')
maregrafo["Datetime"] = pd.to_datetime(maregrafo["Datetime"]).astype('datetime64[ms, UTC]')

In [13]:
cattalini_maregrafo = maregrafo[maregrafo["Tábua"] == "Cattalini"]
cobras_maregrafo = maregrafo[maregrafo["Tábua"] == "Ilha das Cobras"]
felix_maregrafo = maregrafo[maregrafo["Tábua"] == "Ponta do Félix"]
galheta_maregrafo = maregrafo[maregrafo["Tábua"] == "Canal da Galheta"]
jerere_maregrafo = maregrafo[maregrafo["Tábua"] == "Jererê"]
caisoeste_maregrafo = maregrafo[maregrafo["Tábua"] == "Cais Oeste"]
caisleste_maregrafo = maregrafo[maregrafo["Tábua"] == "Cais Leste"]

In [14]:
cattalini_maregrafo.drop(["Tábua"], axis=1, inplace=True)
cobras_maregrafo.drop(["Tábua"], axis=1, inplace=True)
felix_maregrafo.drop(["Tábua"], axis=1, inplace=True)
galheta_maregrafo.drop(["Tábua"], axis=1, inplace=True)
jerere_maregrafo.drop(["Tábua"], axis=1, inplace=True)
caisoeste_maregrafo.drop(["Tábua"], axis=1, inplace=True)
caisleste_maregrafo.drop(["Tábua"], axis=1, inplace=True)

# Dados Maré Astronomica

In [15]:
mare_astronomica = pd.read_csv("/content/drive/MyDrive/Monografia/Dados Paranaguá/Aglutinados/Mare Astronomica/Mare Astronomica.csv")

In [16]:
mare_astronomica.replace({
    "Canal Da Galheta" : "Canal da Galheta",
    'Ilha Das Cobras' : 'Ilha das Cobras',
    'Ponta Felix' : 'Ponta do Félix',
}, inplace=True)

In [17]:
mare_astronomica.columns = ["Tábua", "Datetime", "Altura"]
mare_astronomica["Altura"] = mare_astronomica["Altura"].replace(",", ".", regex=True)
mare_astronomica["Altura"] = mare_astronomica["Altura"].astype(float)
mare_astronomica.loc[mare_astronomica["Datetime"].str.len() == 10, "Datetime"] += " 00:00"

In [18]:
mare_astronomica['Datetime'] = pd.to_datetime(mare_astronomica['Datetime'], format='mixed', utc=True).dt.round('ms')
mare_astronomica['Datetime'] = pd.to_datetime(mare_astronomica['Datetime']).astype('datetime64[ms, UTC]')

In [19]:
oeste_astronomica = mare_astronomica[mare_astronomica["Tábua"] == "Cais Oeste"]
galheta_astronomica = mare_astronomica[mare_astronomica["Tábua"] == "Canal da Galheta"]
cobras_astronomica = mare_astronomica[mare_astronomica["Tábua"] == "Ilha das Cobras"]
felix_astronomica = mare_astronomica[mare_astronomica["Tábua"] == "Ponta do Félix"]

In [20]:
galheta_astronomica.drop(['Tábua'], axis=1, inplace=True)
oeste_astronomica.drop(['Tábua'], axis=1, inplace=True)
cobras_astronomica.drop(['Tábua'], axis=1, inplace=True)
felix_astronomica.drop(['Tábua'], axis=1, inplace=True)

# Harmônicos

In [21]:
# Definindo datas de início e fim da série temporal
start_date = pd.to_datetime("2019-04-25 19:30:00+00:00")
end_date = pd.to_datetime("2021-12-08 15:00:00+00:00")

# Data de referência para o início dos tempos
reference_date = pd.to_datetime("1995-04-01 03:23:00+00:00")

# Criando o vetor de tempo a cada 10 minutos no intervalo especificado
time = pd.date_range(start=start_date, end=end_date, freq='10min')

# Convertendo para tempo relativo em dias desde a data de referência
time_hours = (time - reference_date) / pd.Timedelta(hours=1)

# Componentes harmônicas e propriedades
cp = [
    'Q1',   'O1',  'M10', 'K1',
    'MU2',  'N2',  'M2', 'L2',
    'S2',  'MO3',  'M3', 'MK3',
    'MN4',  'M4', 'MS4', 'Z0'
]

# Amplitudes em cm e conversão para metros
amp = np.array([3.3, 11.1, 1.6, 7.6, 4.0, 7.4, 49.1, 2.9, 33.3, 10.5, 16.2, 5.9, 6.7, 16.0, 6.7, 90]) / 100

# Fases em graus e conversão para radianos
phase = np.array([57, 80, 273, 138, 144, 167, 96, 105, 101, 61, 262, 152, 221, 272, 356, 0])
phase_rad = np.radians(phase)

# Obtendo os componentes e frequências do UTide
components = ut_constants.const['name']
frequencies = ut_constants.const['freq']

# Selecionando índices das componentes desejadas
index = [np.where(components == name)[0][0] for name in cp]

# Frequências correspondentes às componentes selecionadas
freq = frequencies[index]

# Calculando a altura da superfície do mar (SSH)
ssh = np.zeros_like(time_hours)
for i in range(len(cp)):
    ssh += amp[i] * np.cos(2 * np.pi * freq[i] * time_hours + phase_rad[i])

# Criando o dataframe com Datetime e altura prevista
porto_ssh = pd.DataFrame({'Datetime': time, 'SSH [m]': ssh})

# Exibindo os primeiros registros do dataframe
porto_ssh.head()

,Datetime,SSH [m]
0,2019-04-25 19:30:00+00:00,0.331453
1,2019-04-25 19:40:00+00:00,0.335344
2,2019-04-25 19:50:00+00:00,0.345919
3,2019-04-25 20:00:00+00:00,0.361391
4,2019-04-25 20:10:00+00:00,0.380047


# Crop de Datetimes

In [22]:
data_list = [
    odas_corrente,
    cattalini_corrente,
    odas_meteorologia,
    cattalini_meteorologia,
    cattalini_maregrafo,
    cobras_maregrafo,
    felix_maregrafo,
    galheta_maregrafo,
    jerere_maregrafo,
    caisoeste_maregrafo,
    caisleste_maregrafo,
    oeste_astronomica,
    galheta_astronomica,
    cobras_astronomica,
    felix_astronomica,
    porto_ssh
]

data_name = [
    "ODAS             Correntômetro ",
    "Cattalini        Correntômetro ",
    "ODAS             Meteorologia  ",
    "Cattalini        Meteorologia  ",
    "Cattalini        Maregrafo     ",
    "Ilha das Cobras  Maregrafo     ",
    "Ponta do Félix   Maregrafo     ",
    "Canal da Galheta Maregrafo     ",
    "Jererê           Maregrafo     ",
    "Cais Oeste       Maregrafo     ",
    "Cais Leste       Maregrafo     ",
    "Cais Oeste       Astronomica   ",
    "Canal da Galheta Astronomica   ",
    "Ilha das Cobras  Astronomica   ",
    "Ponta do Félix   Astronomica   ",
    "Porto Paranaguá  Harmônico     ",
]

In [23]:
for i in range(len(data_list)):
    print(i, '\t', data_name[i])
    print("MIN :\t", data_list[i]['Datetime'].min())
    print("MAX :\t", data_list[i]['Datetime'].max())
    print()

0 	 ODAS             Correntômetro 
MIN :	 2019-04-20 15:50:00+00:00
MAX :	 2021-12-08 15:00:00+00:00

1 	 Cattalini        Correntômetro 
MIN :	 2019-04-25 19:30:00+00:00
MAX :	 2022-06-28 14:35:00+00:00

2 	 ODAS             Meteorologia  
MIN :	 2019-04-20 15:50:00+00:00
MAX :	 2022-06-28 13:50:00+00:00

3 	 Cattalini        Meteorologia  
MIN :	 2019-04-25 19:40:00+00:00
MAX :	 2022-06-28 11:58:00+00:00

4 	 Cattalini        Maregrafo     
MIN :	 2019-04-25 19:30:00+00:00
MAX :	 2022-08-22 13:50:00+00:00

5 	 Ilha das Cobras  Maregrafo     
MIN :	 2019-12-23 09:41:17+00:00
MAX :	 2022-08-22 10:21:20+00:00

6 	 Ponta do Félix   Maregrafo     
MIN :	 2015-01-01 00:00:47+00:00
MAX :	 2022-08-22 11:16:28+00:00

7 	 Canal da Galheta Maregrafo     
MIN :	 2015-08-04 16:06:17+00:00
MAX :	 2022-08-22 09:45:00+00:00

8 	 Jererê           Maregrafo     
MIN :	 2020-11-12 13:58:07+00:00
MAX :	 2022-08-22 11:37:00+00:00

9 	 Cais Oeste       Maregrafo     
MIN :	 2015-08-04 16:00:16+00:00
MAX 

Para simplificação do TCC e ter features interessantes, vou excluir os dados que estão muito fora do período de crop.

Vou focar também nos dados do Cattalini, Bóia ODAS e Porto Cais Oeste

In [24]:
# Definindo o intervalo de datas
start_date = pd.to_datetime("2019-04-25 19:30:00+00:00")
end_date = pd.to_datetime("2021-12-08 15:00:00+00:00")

odas_corrente = odas_corrente[(odas_corrente['Datetime'] >= start_date) & (odas_corrente['Datetime'] <= end_date)]
cattalini_corrente = cattalini_corrente[(cattalini_corrente['Datetime'] >= start_date) & (cattalini_corrente['Datetime'] <= end_date)]

odas_meteorologia = odas_meteorologia[(odas_meteorologia['Datetime'] >= start_date) & (odas_meteorologia['Datetime'] <= end_date)]
cattalini_meteorologia = cattalini_meteorologia[(cattalini_meteorologia['Datetime'] >= start_date) & (cattalini_meteorologia['Datetime'] <= end_date)]

cattalini_maregrafo = cattalini_maregrafo[(cattalini_maregrafo['Datetime'] >= start_date) & (cattalini_maregrafo['Datetime'] <= end_date)]
caisoeste_maregrafo = caisoeste_maregrafo[(caisoeste_maregrafo['Datetime'] >= start_date) & (caisoeste_maregrafo['Datetime'] <= end_date)]

oeste_astronomica = oeste_astronomica[(oeste_astronomica['Datetime'] >= start_date) & (oeste_astronomica['Datetime'] <= end_date)]

porto_ssh = porto_ssh[(porto_ssh['Datetime'] >= start_date) & (porto_ssh['Datetime'] <= end_date)]

In [25]:
odas_corrente.sort_values(by="Datetime", inplace=True)
odas_corrente.reset_index(drop=True, inplace=True)

cattalini_corrente.sort_values(by="Datetime", inplace=True)
cattalini_corrente.reset_index(drop=True, inplace=True)

odas_meteorologia.sort_values(by="Datetime", inplace=True)
odas_meteorologia.reset_index(drop=True, inplace=True)

cattalini_meteorologia.sort_values(by="Datetime", inplace=True)
cattalini_meteorologia.reset_index(drop=True, inplace=True)

cattalini_maregrafo.sort_values(by="Datetime", inplace=True)
cattalini_maregrafo.reset_index(drop=True, inplace=True)

caisoeste_maregrafo.sort_values(by="Datetime", inplace=True)
caisoeste_maregrafo.reset_index(drop=True, inplace=True)

oeste_astronomica.sort_values(by="Datetime", inplace=True)
oeste_astronomica.reset_index(drop=True, inplace=True)

porto_ssh.sort_values(by="Datetime", inplace=True)
porto_ssh.reset_index(drop=True, inplace=True)

# Conversão para S.I.

In [26]:
odas_corrente["Velocidade 1 [m/s]"] = odas_corrente["Velocidade 1"] / 1.943844
odas_corrente["Direção 1 [°N]"] = odas_corrente["Direção 1"]

odas_corrente["Velocidade 2 [m/s]"] = odas_corrente["Velocidade 2"] / 1.943844
odas_corrente["Direção 2 [°N]"] = odas_corrente["Direção 2"]

odas_corrente["Velocidade 3 [m/s]"] = odas_corrente["Velocidade 3"] / 1.943844
odas_corrente["Direção 3 [°N]"] = odas_corrente["Direção 3"]

odas_corrente["Velocidade 4 [m/s]"] = odas_corrente["Velocidade 4"] / 1.943844
odas_corrente["Direção 4 [°N]"] = odas_corrente["Direção 4"]

odas_corrente["Velocidade 5 [m/s]"] = odas_corrente["Velocidade 5"] / 1.943844
odas_corrente["Direção 5 [°N]"] = odas_corrente["Direção 5"]

odas_corrente["Velocidade 6 [m/s]"] = odas_corrente["Velocidade 6"] / 1.943844
odas_corrente["Direção 6 [°N]"] = odas_corrente["Direção 6"]

odas_corrente.drop(["Velocidade 1", "Velocidade 2", "Velocidade 3",
                    "Velocidade 4", "Velocidade 5", "Velocidade 6",
                    "Direção 1", "Direção 2", "Direção 3",
                    "Direção 4", "Direção 5", "Direção 6",],
                   axis=1, inplace=True)

In [27]:
cattalini_corrente["Velocidade 1 [m/s]"] = cattalini_corrente["Velocidade 1"] / 1.943844
cattalini_corrente["Direção 1 [°N]"] = cattalini_corrente["Direção 1"]

cattalini_corrente["Velocidade 2 [m/s]"] = cattalini_corrente["Velocidade 2"] / 1.943844
cattalini_corrente["Direção 2 [°N]"] = cattalini_corrente["Direção 2"]

cattalini_corrente["Velocidade 3 [m/s]"] = cattalini_corrente["Velocidade 3"] / 1.943844
cattalini_corrente["Direção 3 [°N]"] = cattalini_corrente["Direção 3"]

cattalini_corrente.drop(["Velocidade 1", "Velocidade 2", "Velocidade 3",
                    "Velocidade 4", "Velocidade 5", "Velocidade 6",
                    "Direção 1", "Direção 2", "Direção 3",
                    "Direção 4", "Direção 5", "Direção 6",],
                   axis=1, inplace=True)

In [28]:
odas_meteorologia['Velocidade do Vento [m/s]'] = odas_meteorologia['Velocidade do Vento'] / 1.943844
odas_meteorologia['Rajada de Vento [m/s]'] = odas_meteorologia['Rajada de Vento'] / 1.943844
odas_meteorologia['Direção do Vento [°N]'] = odas_meteorologia['Direção do Vento']

odas_meteorologia.drop(['Velocidade do Vento', 'Rajada de Vento',
                        'Direção do Vento', 'Setor do Vento'],
                       axis=1, inplace=True)

In [29]:
cattalini_meteorologia['Velocidade do Vento [m/s]'] = cattalini_meteorologia['Velocidade do Vento'] / 1.943844
cattalini_meteorologia['Rajada de Vento [m/s]'] = cattalini_meteorologia['Rajada de Vento'] / 1.943844
cattalini_meteorologia['Direção do Vento [°N]'] = cattalini_meteorologia['Direção do Vento']

cattalini_meteorologia.drop(['Velocidade do Vento', 'Rajada de Vento',
                        'Direção do Vento', 'Setor do Vento'],
                       axis=1, inplace=True)

In [30]:
cattalini_maregrafo['Altura [m]'] = cattalini_maregrafo['Altura']
caisoeste_maregrafo['Altura [m]'] = caisoeste_maregrafo['Altura']

cattalini_maregrafo.drop(['Altura'], axis=1, inplace=True)
caisoeste_maregrafo.drop(['Altura'], axis=1, inplace=True)

In [31]:
oeste_astronomica['Altura [m]'] = oeste_astronomica['Altura'] / 100
oeste_astronomica.drop(['Altura'], axis=1, inplace=True)

# Exportação CSV

In [32]:
os.chdir("/content/drive/MyDrive/Monografia/Dados Paranaguá/P2MOD/CSV")

odas_corrente.to_csv("ODAS_Corrente.csv", index=False)
cattalini_corrente.to_csv("Cattalini_Corrente.csv", index=False)

odas_meteorologia.to_csv("ODAS_Meteorologia.csv", index=False)
cattalini_meteorologia.to_csv("Cattalini_Meteorologia.csv", index=False)

cattalini_maregrafo.to_csv("Cattalini_Maregrafo.csv", index=False)
caisoeste_maregrafo.to_csv("Porto_Maregrafo.csv", index=False)

oeste_astronomica.to_csv("Porto_Astronomica.csv", index=False)

porto_ssh.to_csv("Porto_Harmonicos.csv", index=False)

# Exportação PICKLE

In [33]:
os.chdir("/content/drive/MyDrive/Monografia/Dados Paranaguá/P2MOD/PICKLE")

odas_corrente.to_pickle("ODAS_Corrente.pkl")
cattalini_corrente.to_pickle("Cattalini_Corrente.pkl")

odas_meteorologia.to_pickle("ODAS_Meteorologia.pkl")
cattalini_meteorologia.to_pickle("Cattalini_Meteorologia.pkl")

cattalini_maregrafo.to_pickle("Cattalini_Maregrafo.pkl")
caisoeste_maregrafo.to_pickle("Porto_Maregrafo.pkl")

oeste_astronomica.to_pickle("Porto_Astronomica.pkl")

porto_ssh.to_pickle("Porto_Harmonico.pkl")

# Exportação PARQUET

In [34]:
os.chdir("/content/drive/MyDrive/Monografia/Dados Paranaguá/P2MOD/PARQUET")

odas_corrente.to_parquet("ODAS_Corrente.parquet")
cattalini_corrente.to_parquet("Cattalini_Corrente.parquet")

odas_meteorologia.to_parquet("ODAS_Meteorologia.parquet")
cattalini_meteorologia.to_parquet("Cattalini_Meteorologia.parquet")

cattalini_maregrafo.to_parquet("Cattalini_Maregrafo.parquet")
caisoeste_maregrafo.to_parquet("Porto_Maregrafo.parquet")

oeste_astronomica.to_parquet("Porto_Astronomica.parquet")

porto_ssh.to_parquet("Porto_Harmonico.parquet")